In [1]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
from net import Net
from torchsampler import ImbalancedDatasetSampler

In [2]:
# Use CUDA if possible
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")

In [3]:
train_dir = './train_images'    # folder containing training images
test_dir = './test_images'    # folder containing test images

transform = transforms.Compose(
    [transforms.Grayscale(),   # transforms to gray-scale (1 input channel)
     transforms.ToTensor(),    # transforms to Torch tensor (needed for PyTorch)
     transforms.Normalize(mean=(0.5,),std=(0.5,))]) # subtracts mean (0.5) and devides by standard deviation (0.5) -> resulting values in (-1, +1)

In [4]:
# Define two pytorch datasets (train/test) 
train_data = torchvision.datasets.ImageFolder(train_dir, transform=transform)
test_data = torchvision.datasets.ImageFolder(test_dir, transform=transform)

valid_size = 0.2   # proportion of validation set (80% train, 20% validation)
batch_size = 32    

# Define randomly the indices of examples to use for training and for validation
num_train = len(train_data)
indices_train = list(range(num_train))
np.random.shuffle(indices_train)
split_tv = int(np.floor(valid_size * num_train))
train_new_idx, valid_idx = indices_train[split_tv:],indices_train[:split_tv]


In [5]:
# Define two "samplers" that will randomly pick examples from the unbalanced training and validation set
train_sampler = ImbalancedDatasetSampler(train_data, train_new_idx)
valid_sampler = ImbalancedDatasetSampler(train_data, valid_idx)

# Dataloaders (take care of loading the data from disk, batch by batch, during training)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=train_sampler, num_workers=4)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=4)

classes = ('noface','face')  # indicates that "1" means "face" and "0" non-face (only used for display)

In [6]:
print ("train: ", len(train_sampler))
print ("test: ", len(test_data))
count =0

for i in range(0,len(train_sampler)):
    if train_sampler._get_label(train_data,i) == 0:
        count +=1
print(count)


train:  73376
test:  7628
26950


In [7]:
net = Net()
net = net.to(device)
n_epochs = 32

optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

In [8]:
classification_map = {"TP" : 0,
                      "FP" : 0,
                      "TN" : 0,
                      "FN" : 0}

In [9]:
# Training 
running_loss =0
# loop over epochs: one epoch = one pass through the whole training dataset
for epoch in range(1, n_epochs+1):  
#   loop over iterations: one iteration = 1 batch of examples
    running_loss =0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() # zero the gradient buffers
        output = net(data)
        loss = criterion(output, target)
        running_loss +=loss
        loss.backward()
        optimizer.step() # Does the update
    print ('epoch: %d, running_loss: %5.7f' % (epoch,running_loss))  

epoch: 1, running_loss: 168.4761353
epoch: 2, running_loss: 45.3047829
epoch: 3, running_loss: 32.8437576
epoch: 4, running_loss: 25.0212345
epoch: 5, running_loss: 21.7288227
epoch: 6, running_loss: 18.1002312
epoch: 7, running_loss: 15.6587963
epoch: 8, running_loss: 13.9877157
epoch: 9, running_loss: 13.2276754
epoch: 10, running_loss: 12.8625126
epoch: 11, running_loss: 11.3820152
epoch: 12, running_loss: 13.2031689
epoch: 13, running_loss: 10.7207289
epoch: 14, running_loss: 10.9330978
epoch: 15, running_loss: 9.3848009
epoch: 16, running_loss: 9.4787989
epoch: 17, running_loss: 8.9973955
epoch: 18, running_loss: 10.2980118
epoch: 19, running_loss: 7.9035530
epoch: 20, running_loss: 7.2286773
epoch: 21, running_loss: 8.0139933
epoch: 22, running_loss: 7.0642419
epoch: 23, running_loss: 4.7437434
epoch: 24, running_loss: 8.2998848
epoch: 25, running_loss: 7.0300188
epoch: 26, running_loss: 7.4751620
epoch: 27, running_loss: 7.9221659
epoch: 28, running_loss: 7.5973816
epoch: 29, ru

In [10]:
classification_map = {"TP" : 0,
                      "FP" : 0,
                      "TN" : 0,
                      "FN" : 0}

correct = 0
total = 0
count = 0
with torch.no_grad():
    for data in test_loader:
        count+=1
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        
        for i in range(0,len(labels)):
            if predicted[i].item() == labels[i].item():
                if predicted[i].item() == 1:
                    classification_map["TP"] +=1
                else:
                    classification_map["TN"] +=1
            elif predicted[i].item() == 1:
                classification_map["FP"] +=1
            else: 
                classification_map["FN"] +=1

        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %5.6f %%' % (
    100 * correct / total))

print(count)

Accuracy of the network on the 10000 test images: 95.228107 %
239


In [11]:

print (classification_map["TP"])
print (classification_map["TN"])
print (classification_map["FP"])
print (classification_map["FN"])

stats_map = {
            "Specificity" : float(classification_map["TN"]) / float(classification_map["TN"] + classification_map["FP"]),
            "Recall" : float(classification_map["TP"]) / float(classification_map["TP"] + classification_map["FN"]),
            "Precision" : float(classification_map["TP"]) / float(classification_map["TP"] + classification_map["FP"]),
            "Accuracy" : float(classification_map["TP"] + classification_map["TN"]) / float(classification_map["TP"] + classification_map["TN"] + classification_map["FP"] + classification_map["FN"])
        }
stats_map["F-score"] = 2.0 / float((1.0 / float(stats_map["Precision"])) + (1.0 / float(stats_map["Recall"])))

for key, value in stats_map.items():
    print(key, ": ", value)

478
6786
45
319
Specificity :  0.9934123847167325
Recall :  0.5997490589711418
Precision :  0.9139579349904398
Accuracy :  0.9522810697430519
F-score :  0.7242424242424242
